In [ ]:
# uncomment the below lines of code and run once to install packages needed for this lab
#!pip install snowflake-snowpark-python pandas notebook scikit-learn cachetools pyarrow==10.0.1
#!pip install azure-ai-ml
#!pip install mltable

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
# # Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="<azure subscription number>",
    resource_group_name="<resource group name>",
    workspace_name="<azureml workspace name>",
)

In [3]:
import mltable
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get the latest version of the data asset
# Note: The version was set in the previous code cell.
data_asset = ml_client.data.get(name="purchasehistory", version="1")
# create a table
tbl = mltable.load(f"azureml:/{data_asset.id}")
df = tbl.to_pandas_dataframe()
df['MED_AGE'] = df['MED_AGE'].apply(str)
df

,ID,RECENT_PURCHASES,ZIP,MED_AGE
0,1,"1 Gallon Milk, 24 oz Bread, Dozen Eggs",35404,29
1,2,"16 oz Toothpaste, 12 oz Shampoo, 8 oz Soap",33956,65
2,3,"5 lb Potatoes, 3 lb Onions, 1 lb Carrots",59703,59
3,4,"2 lb Chicken, 1 lb Beef, 0.75 lb Salmon",73043,59
4,5,"18 oz Cereal, 6 oz Yogurt, 1.5 oz Granola Bars",75412,56
5,6,"16 oz Pasta, 24 oz Tomato Sauce, 3 cloves Garlic",15467,53
6,7,"Bunch of Bananas, 1 lb Grapes, 16 oz Strawberries",75217,28
7,8,"8 oz Chips, 16 oz Salsa, 12 oz Guacamole, 10 c...",65622,46
8,9,"6 Rolls Paper Towels, 12 Rolls Toilet Paper, 1...",60546,42
9,10,"1.5 qt Ice Cream, 12 inch Frozen Pizza, 16 oz ...",1002,23


In [4]:
import pandas as pd
import requests
import json

# Azure OpenAI API endpoint URL
endpoint_url = "<flow endpoint>"

# API key or authentication token for accessing the endpoint
api_key = "<flow endpoint key>"

# Headers for the HTTP request
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

response_data = []

# Iterate through rows and make requests
for index, row in df.iterrows():
    input_purchases = row["RECENT_PURCHASES"]
    input_age = row["MED_AGE"]

    # Construct payload using the input text
    payload = {
        "purchase_history": input_purchases,
        "med_age": input_age,
        "max_tokens": 100
    }

    # Convert payload to JSON and encode as bytes
    payload_bytes = json.dumps(payload).encode("utf-8")

    # Send the POST request
    response = requests.post(endpoint_url, data=payload_bytes, headers=headers)

    # Process the response
    if response.status_code == 200:
        result = response.json()
        generated_text = result.get("output", "No generated text found")
        response_data.append({"Input": input_purchases, "Generated": generated_text})
    else:
        print(f"Request for '{input_purchases}' failed with status code:", response.status_code)

# Create a DataFrame from the response data list
response_df = pd.DataFrame(response_data)
response_df['ID'] = df['ID']

# Print the resulting DataFrame
response_df

,Input,Generated,ID
0,"1 Gallon Milk, 24 oz Bread, Dozen Eggs",1. Organic Butter\n2. Avocado\n3. Granola,1
1,"16 oz Toothpaste, 12 oz Shampoo, 8 oz Soap",1. Mouthwash\n2. Hair conditioner\n3. Body lotion,2
2,"5 lb Potatoes, 3 lb Onions, 1 lb Carrots",1. Bag of frozen mixed vegetables\n2. Vegetabl...,3
3,"2 lb Chicken, 1 lb Beef, 0.75 lb Salmon",1. Fresh herbs and spices\n2. Organic mixed gr...,4
4,"18 oz Cereal, 6 oz Yogurt, 1.5 oz Granola Bars",1. Multivitamins for seniors\n2. Whole wheat t...,5
5,"16 oz Pasta, 24 oz Tomato Sauce, 3 cloves Garlic",1. Parmesan cheese \n2. Chianti wine \n3. Arti...,6
6,"Bunch of Bananas, 1 lb Grapes, 16 oz Strawberries",1. Watermelon\n2. Blueberries\n3. Greek Yogurt,7
7,"8 oz Chips, 16 oz Salsa, 12 oz Guacamole, 10 c...",1. 16 oz Beer\n2. 1 lb Ground Beef\n3. 8 ct Ho...,8
8,"6 Rolls Paper Towels, 12 Rolls Toilet Paper, 1...",1. Disinfectant Wipes\n2. Hand Soap\n3. Air Fr...,9
9,"1.5 qt Ice Cream, 12 inch Frozen Pizza, 16 oz ...",1. Frozen Chicken Nuggets \n2. Frozen French F...,10


In [5]:
from snowflake.snowpark import Session
connection_parameters = {
    "account": "<snowflake account>", # e.g. SN00102.central-us.azure
    "user": "<username>", 
    "password": "<password>",
    "role": "ACCOUNTADMIN",
    "database": "retail_db",
    "schema": "PUBLIC",
    "warehouse": "SMALL_WH"
    }
session = Session.builder.configs(connection_parameters).create()
session.write_pandas(response_df, table_name='NBI_PROMO', auto_create_table=True, overwrite=True)